Improve Accuracy from 0.49 to something higher. 

In [1]:
# Do not drop NAs (try to do some better imputation + var selection)

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression


In [6]:
train = pd.read_csv('train.csv')

In [7]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Don't need PassengerId, Name, Ticket, Cabin

In [8]:
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)

In [10]:
# Get dummies - one hot encoding
gender_dummies = pd.get_dummies(train['Sex'])
class_dummies = pd.get_dummies(train['Pclass'])
embark_dummies = pd.get_dummies(train['Embarked'])
train = pd.concat([train, gender_dummies, class_dummies, embark_dummies], axis = 1)
train = train.drop(['female', 1, 'C', 'Pclass', 'Sex', 'Embarked'], axis = 1)
# Impute with simple imputer
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
train = pd.DataFrame(my_imputer.fit_transform(train), columns = train.columns)

In [15]:
train.head()

,Survived,Age,SibSp,Parch,Fare,male,2,3,Q,S
0,0.0,22.0,1.0,0.0,7.2500,1.0,0.0,1.0,0.0,1.0
1,1.0,38.0,1.0,0.0,71.2833,0.0,0.0,0.0,0.0,0.0
2,1.0,26.0,0.0,0.0,7.9250,0.0,0.0,1.0,0.0,1.0
3,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,0.0,0.0,1.0
4,0.0,35.0,0.0,0.0,8.0500,1.0,0.0,1.0,0.0,1.0


d:\python\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
d:\python\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [18]:
train['Age'].isnull().sum()

0

,Age,SibSp,Parch,Fare,male,2,3,Q,S
0,22.000000,1.0,0.0,7.2500,1.0,0.0,1.0,0.0,1.0
1,38.000000,1.0,0.0,71.2833,0.0,0.0,0.0,0.0,0.0
2,26.000000,0.0,0.0,7.9250,0.0,0.0,1.0,0.0,1.0
3,35.000000,1.0,0.0,53.1000,0.0,0.0,0.0,0.0,1.0
4,35.000000,0.0,0.0,8.0500,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
886,27.000000,0.0,0.0,13.0000,1.0,1.0,0.0,0.0,1.0
887,19.000000,0.0,0.0,30.0000,0.0,0.0,0.0,0.0,1.0
888,29.699118,1.0,2.0,23.4500,0.0,0.0,1.0,0.0,1.0
889,26.000000,0.0,0.0,30.0000,1.0,0.0,0.0,0.0,0.0


In [34]:
Y = train['Survived']
X = train.iloc[:,1:]

In [46]:
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size = 0.2)

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [84]:
model = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)
model.fit(train_X, train_Y)
predictions = model.predict(val_X)

d:\python\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
d:\python\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [72]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [85]:
print(classification_report(val_Y, predictions))

              precision    recall  f1-score   support

         0.0       0.81      0.91      0.86       109
         1.0       0.82      0.67      0.74        70

    accuracy                           0.82       179
   macro avg       0.82      0.79      0.80       179
weighted avg       0.82      0.82      0.81       179



# Test Data

In [99]:
test = pd.read_csv('test.csv')

In [100]:
test_data = test.copy()

In [91]:
# Run the exact same transformations on test as we did on train
test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)

In [92]:
# Get dummies - one hot encoding
gender_dummies = pd.get_dummies(test['Sex'])
class_dummies = pd.get_dummies(test['Pclass'])
embark_dummies = pd.get_dummies(test['Embarked'])
test = pd.concat([test, gender_dummies, class_dummies, embark_dummies], axis = 1)
test = test.drop(['female', 1, 'C', 'Pclass', 'Sex', 'Embarked'], axis = 1)
# Impute with simple imputer
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
test = pd.DataFrame(my_imputer.fit_transform(test), columns = test.columns)

d:\python\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
d:\python\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [94]:
final_pred = model.predict(test)

d:\python\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [117]:
final_pred = pd.Series(final_pred)

In [122]:
submission = pd.concat([test_data, final_pred], axis = 1)

In [125]:
submission = submission[['PassengerId', 0]]

In [127]:
submission.rename(columns = {0:'Survived'}, inplace = True)

C:\Users\kamil\AppData\Local\Temp\ipykernel_2668\2010006363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission.rename(columns = {0:'Survived'}, inplace = True)


In [129]:
submission.to_csv('Submission.csv', index = False)